In [1]:
import pandas as pd

# Load data from Excel file
file_path = './data/Subset-input data for Contact1 - Centerbridge - 3k 2.xlsx'
df = pd.read_excel(file_path)

# Display the first few rows of the DataFrame

df.sort_values(by='LastName', inplace=True)
print(df.head())


     Id FirstName  LastName            E-mail Address  Company  Job Title
0   659  Fernando   2Forces                       NaN      NaN        NaN
1   539      Todd  Abbrecht         tabbrecht@thl.com      NaN        NaN
2  2184     Peter    Abdill   peter.abdill@moodys.com      NaN        NaN
3  1782    Faysal  Abillama  faysal.abillama@hsbc.com      NaN        NaN
4  1573     Salon     Above                       NaN      NaN        NaN


In [ ]:
df.info()

In [2]:
from Levenshtein import ratio

def is_similar(name1, name2, threshold=0.55):
    return ratio(name1, name2) >= threshold


In [3]:
def group_by_last_name(df, threshold=0.85):
    groups = []
    for index, row in df.iterrows():
        last_name = str(row['LastName'])  # Ensure last_name is a string
        found_group = False
        for group in groups:
            # Check if the first character of the last name is the same
            if last_name[0] == str(group[0]['LastName'])[0]:
                if is_similar(last_name, str(group[0]['LastName']), threshold):
                    group.append(row)
                    found_group = True
                    break
        if not found_group:
            groups.append([row])
    return groups

In [ ]:
#old logic
def group_by_last_name(df, threshold=0.8):
    groups = []
    for index, row in df.iterrows():
        last_name = row['LastName']
        found_group = False
        for group in groups:
            if is_similar(last_name, group[0]['LastName'], threshold):
                group.append(row)
                found_group = True
                break
        if not found_group:
            groups.append([row])
    return groups


In [ ]:
# new logic with add first letter 
def refine_by_first_name(groups, threshold=0.65):
    refined_groups = []
    for group in groups:
        refined_group = []
        for contact in group:
            first_name = str(contact.get('FirstName', ''))  # Ensure first_name is a string
            if not first_name:
                continue
            found_subgroup = False
            for subgroup in refined_group:
                if first_name[0] == str(subgroup[0]['FirstName'])[0] or len(first_name) == 1:
                    if is_similar(first_name, str(subgroup[0]['FirstName']), threshold) or len(first_name) == 1:
                        subgroup.append(contact)
                        found_subgroup = True
                        break
            if not found_subgroup:
                refined_group.append([contact])
        refined_groups.append(refined_group)
    return refined_groups

In [ ]:
def refine_by_first_name(groups, threshold=0.65):
    refined_groups = []
    for group in groups:
        refined_group = []
        for contact in group:
            first_name = str(contact['FirstName'])  # Ensure first_name is a string
            found_subgroup = False
            for subgroup in refined_group:
                # Check if the first letter of the first name is the same
                if first_name[0] == str(subgroup[0]['FirstName'])[0]:
                    # Apply similarity check
                    if is_similar(first_name, str(subgroup[0]['FirstName']), threshold):
                        subgroup.append(contact)
                        found_subgroup = True
                        break
            if not found_subgroup:
                refined_group.append([contact])
        refined_groups.append(refined_group)
    return refined_groups

In [ ]:
#without_First_name_first_letter_Matching
def refine_by_first_name(groups, threshold=0.68):
    refined_groups = []
    for group in groups:
        refined_group = []
        for contact in group:
            first_name = contact['FirstName']
            found_subgroup = False
            for subgroup in refined_group:
                if is_similar(first_name, subgroup[0]['FirstName'], threshold) or (len(first_name) == 1 and first_name == subgroup[0]['FirstName'][0]):
                    subgroup.append(contact)
                    found_subgroup = True
                    break
            if not found_subgroup:
                refined_group.append([contact])
        refined_groups.append(refined_group)
    return refined_groups

In [ ]:
#old One
def refine_by_first_name(groups, threshold=0.68):
    refined_groups = []
    for group in groups:
        refined_group = []
        for contact in group:
            first_name = contact['FirstName']
            found_subgroup = False
            for subgroup in refined_group:
                if is_similar(first_name, subgroup[0]['FirstName'], threshold):
                    subgroup.append(contact)
                    found_subgroup = True
                    break
            if not found_subgroup:
                refined_group.append([contact])
        refined_groups.append(refined_group)
    return refined_groups


In [4]:
def assign_ids(refined_groups):
    group_id = 1
    subgroup_id = 1  # Initialize a global counter for SubgroupID
    result = []
    for group in refined_groups:
        for subgroup in group:
            is_duplicate = len(subgroup) > 1
            for contact in subgroup:
                if is_duplicate:
                    contact['GroupID'] = group_id
                    contact['SubgroupID'] = subgroup_id
                else:
                    contact['GroupID'] = group_id
                    contact['SubgroupID'] = 9999999  # No subgroup ID for unique records
                contact['IsDuplicate'] = is_duplicate
                result.append(contact)
            if is_duplicate:
                subgroup_id += 1  # Increment the global SubgroupID counter only for duplicates
        group_id += 1
    return result

In [ ]:
def assign_ids(refined_groups):
    group_id = 1
    subgroup_id = 1  # Initialize a global counter for SubgroupID
    result = []
    for group in refined_groups:
        for subgroup in group:
            for contact in subgroup:
                contact['GroupID'] = group_id
                contact['SubgroupID'] = subgroup_id
                contact['IsDuplicate'] = len(subgroup) > 1
                result.append(contact)
            subgroup_id += 1  # Increment the global SubgroupID counter
        group_id += 1
    return result

In [ ]:
#old
def assign_ids(refined_groups):
    group_id = 1
    result = []
    for group in refined_groups:
        subgroup_id = 1
        for subgroup in group:
            for contact in subgroup:
                contact['GroupID'] = group_id
                contact['SubgroupID'] = subgroup_id
                contact['IsDuplicate'] = len(subgroup) > 1
                result.append(contact)
            subgroup_id += 1
        group_id += 1
    return result


In [5]:
def refine_by_first_name(groups, threshold=0.65):
    refined_groups = []
    for group in groups:
        refined_group = []
        single_letter_names = []
        
        for contact in group:
            first_name = str(contact.get('FirstName', ''))  # Ensure first_name is a string
            if not first_name:
                continue
            if len(first_name) == 1:
                single_letter_names.append(contact)
                continue
            
            found_subgroup = False
            for subgroup in refined_group:
                if first_name[0] == str(subgroup[0]['FirstName'])[0]:
                    if is_similar(first_name, str(subgroup[0]['FirstName']), threshold):
                        subgroup.append(contact)
                        found_subgroup = True
                        break
            if not found_subgroup:
                refined_group.append([contact])
        
        # Integrate single-letter names into the appropriate subgroups
        for contact in single_letter_names:
            first_name = str(contact.get('FirstName', ''))
            found_subgroup = False
            for subgroup in refined_group:
                if first_name[0] == str(subgroup[0]['FirstName'])[0]:
                    subgroup.append(contact)
                    found_subgroup = True
                    break
            if not found_subgroup:
                refined_group.append([contact])
        
        refined_groups.append(refined_group)
    
    return refined_groups

In [6]:
broad_groups = group_by_last_name(df)

# Refine by first name
refined_groups = refine_by_first_name(broad_groups)

# Assign unique IDs and flag duplicates
processed_contacts = assign_ids(refined_groups)

# Convert processed contacts back to DataFrame
processed_df = pd.DataFrame(processed_contacts)
print(processed_df.head())


     Id FirstName  LastName            E-mail Address  Company  Job Title  \
0   659  Fernando   2Forces                       NaN      NaN        NaN   
1   539      Todd  Abbrecht         tabbrecht@thl.com      NaN        NaN   
2  2184     Peter    Abdill   peter.abdill@moodys.com      NaN        NaN   
3  1782    Faysal  Abillama  faysal.abillama@hsbc.com      NaN        NaN   
4  1573     Salon     Above                       NaN      NaN        NaN   

   GroupID  SubgroupID  IsDuplicate  
0        1     9999999        False  
1        2     9999999        False  
2        3     9999999        False  
3        4     9999999        False  
4        5     9999999        False  


In [7]:
processed_df.to_csv('./Subset-input data for COntact - Centerbridge - 3k.csv', index=False)

In [10]:
import pandas as pd
import unicodedata
 
# Define a mapping for converting special characters to nearest English characters
def map_special_characters(char):
    special_to_english = {
        'Î': 'I', 'Ï': 'I', 'Ù': 'U', 'Ý': 'Y',
        'à': 'a', 'è': 'e', 'é': 'e', 'ê': 'e', 'ë': 'e',
        'ì': 'i', 'ò': 'o', 'ô': 'o', 'ù': 'u', 'û': 'u',
        'ý': 'y', 'ÿ': 'y', 'ā': 'a', 'ē': 'e', 'ī': 'i',
        'ō': 'o', 'ū': 'u', 'ç': 'c', 'ñ': 'n', 'ğ': 'g',
        # Add more mappings as necessary
    }
    return special_to_english.get(char, char)
 
def clean_name(name, replace_special=True):
    cleaned_name = ''
    
    for char in name:
        char_code = ord(char)
        
        # Check if the character is within the accepted Unicode ranges
        if (
            (0x0041 <= char_code <= 0x005A) or  # A-Z
            (0x0061 <= char_code <= 0x007A) or  # a-z
            (0x00CE <= char_code <= 0x00D6) or  # Î-Ö
            (0x00D9 <= char_code <= 0x00DD) or  # Ù-Ý
            (0x00E0 <= char_code <= 0x00F6) or  # à-ö
            (0x00F9 <= char_code <= 0x00FD) or  # ù-ý
            (char_code == 0x00FF) or  # ÿ
            (0x0100 <= char_code <= 0x0131) or  # Ā-ı
            (0x0134 <= char_code <= 0x0149) or  # Ĵ-õ
            (0x014C <= char_code <= 0x0165) or  # Ō-ť
            (0x0168 <= char_code <= 0x017E)     # Ũ-ž
        ):
            cleaned_name += char
        elif replace_special:
            # Replace special characters with their nearest English equivalents
            cleaned_name += map_special_characters(char)
    
    return cleaned_name
 
def deduplicate_names(df, fname_col='Fname', lname_col='Lname', replace_special=True):
    # Create new columns for cleaned names
    df['Cleaned_Fname'] = ''
    df['Cleaned_Lname'] = ''
    
    # Identify records with special characters
    for index, row in df.iterrows():
        first_name = row[fname_col]
        last_name = row[lname_col]
        
        # Clean names if they contain special characters
        if any(not (0x0041 <= ord(c) <= 0x007A or 0x00CE <= ord(c) <= 0x017E) for c in first_name + last_name):
            cleaned_first = clean_name(first_name, replace_special)
            cleaned_last = clean_name(last_name, replace_special)
            df.at[index, 'Cleaned_Fname'] = cleaned_first
            df.at[index, 'Cleaned_Lname'] = cleaned_last
        else:
            df.at[index, 'Cleaned_Fname'] = first_name
            df.at[index, 'Cleaned_Lname'] = last_name
 
    return df
 
# Example usage:
data = {
    'Fname': ['John', 'Jöhn', 'Anna-Marie', 'José', 'Mårk', 'Elżbieta', 'André', 'Sümmer', 'TóTPφψλη∞', 'Lúcia', 'AαβΓΔΕζ', 'Róbert', 'C@rol', 'J@ne$', '𝒜𝓇𝒾𝒶'],
    'Lname': ['Doe', 'Dœ', 'O\'Connor', 'García', 'Smith', 'Kowalski', 'Tóth', 'Müller', 'Johnson', 'Fernández', 'Brown', 'Leclerc', 'd\'Angelo', 'Smith123', 'Kim']
}
 
df = pd.DataFrame(data)
 
# Clean the DataFrame
cleaned_df = deduplicate_names(df)
 
# Display the cleaned DataFrame
print(cleaned_df[['Fname', 'Lname', 'Cleaned_Fname', 'Cleaned_Lname']])


         Fname      Lname Cleaned_Fname Cleaned_Lname
0         John        Doe          John           Doe
1         Jöhn         Dœ          Jöhn            Dœ
2   Anna-Marie   O'Connor    Anna-Marie      O'Connor
3         José     García          José        García
4         Mårk      Smith          Mårk         Smith
5     Elżbieta   Kowalski      Elżbieta      Kowalski
6        André       Tóth         André          Tóth
7       Sümmer     Müller        Sümmer        Müller
8    TóTPφψλη∞    Johnson     TóTPφψλη∞       Johnson
9        Lúcia  Fernández         Lúcia     Fernández
10     AαβΓΔΕζ      Brown       AαβΓΔΕζ         Brown
11      Róbert    Leclerc        Róbert       Leclerc
12       C@rol   d'Angelo         C@rol      d'Angelo
13       J@ne$   Smith123         J@ne$      Smith123
14        𝒜𝓇𝒾𝒶        Kim          𝒜𝓇𝒾𝒶           Kim


In [16]:
import pandas as pd
import re

# Sample dataset
data = {
    'Complete name': ['Adam Smith'] * 10,
    'Firstname': ['A', '', 'A', 'Adam', 'Adam', 'A', '', 'A', 'Adam', 'Adam'],
    'Last name': ['Smith'] + ['Smith'] * 9,
    'Email': [
        'a.smith@domain.com',
        'adam.smith@domain.com',
        'adam_smith@domain.com',
        'adam-smith@domain.com',
        'asmith@domain.com',
        'adamsmith@domain.com',
        'asmith@domain.com',
        'adam_smith@domain.com',
        'adam-smith@domain.com',
        'smith-adam@domain.com'
    ]
}

# Create DataFrame
df = pd.read_excel("./data/Email extraction.xlsx")

# Function to extract Fname and Lname from email
def extract_names(row):
    email = row['Email']
    first_name = row['Firstname']
    last_name = row['Last Name']

    # Adjusted regex patterns to capture various cases
    patterns = [
        r'(?P<Fname>[A-Za-z])\.(?P<Lname>[A-Za-z]+)',  # Pattern 1: F.Lname
        r'(?P<Fname>[A-Za-z]+)\.(?P<Lname>[A-Za-z]+)',  # Pattern 2: Fname.Lname
        r'(?P<Fname>[A-Za-z]+)_(?P<Lname>[A-Za-z]+)',  # Pattern 3: Fname_Lname
        r'(?P<Fname>[A-Za-z]+)-(?P<Lname>[A-Za-z]+)',  # Pattern 4: Fname-Lname
        r'(?P<Lname>[A-Za-z]+)-(?P<Fname>[A-Za-z]+)',  # Pattern 5: Lname-Fname
        r'(?P<Fname>[A-Za-z])(?P<Lname>[A-Za-z]+)',    # Pattern 6: F.Lname (no separator)
        r'(?P<Lname>[A-Za-z]+)@',                        # Pattern 7: Lname only
        r'(?P<Fname>[A-Za-z]+)@',                        # Pattern 8: Fname only
        r'(?P<Fname>[A-Za-z]+)(?P<Lname>[A-Za-z]+)@',   # Pattern 9: FnameLname without separator
        r'(?P<Fname>[A-Za-z])(?P<Lname>[A-Za-z]+)@domain\.com'  # Pattern 10: F.Lname with domain
    ]

    for pattern in patterns:
        match = re.match(pattern, email)
        if match:
            fname = match.group('Fname')
            lname = match.group('Lname')
            return fname, lname

    return first_name, last_name  # Return defaults if no pattern matches

# Apply the extraction function
df[['Extracted Fname', 'Extracted Lname']] = df.apply(extract_names, axis=1, result_type='expand')

# Display the DataFrame with extracted names
print(df[[ 'Firstname', 'Last Name', 'Email', 'Extracted Fname', 'Extracted Lname']])
df.to_csv("Email_output.csv", index=False)


   Firstname Last Name                         Email Extracted Fname  \
0       Adam         L             landis@lrclaw.com               l   
1          A     Smith         asmith@ioadvisors.com               a   
2          A     Smith     Adam.Smith@butlersnow.com            Adam   
3        Adm     Smith      AdamSmith@butlersnow.com               A   
4          A     Smith      AdamSmith@butlersnow.com               A   
5          A         J       alison.jones@alston.com          alison   
6          A         J       alison.jones@alston.com          alison   
7          A         J        allison.jones@pnfp.com         allison   
8          A      Rose    anedrearose@joelefrank.com               a   
9        NaN    O'Neal       aoneal@pinecrestcap.com               a   
10       NaN     Smith      asmith@houlihansmith.com               a   
11    Arnold         G          agrant@reedsmith.com               a   
12       NaN       NaN      r.oliner@duanemorris.com            

In [31]:
import pandas as pd
import unicodedata
 
def remove_accents(input_str):
    # Normalize the string to its decomposed form (NFKD)
    normalized = unicodedata.normalize('NFKD', input_str)
    # Build a new string without the accent characters (combining marks)
    plain_str = ''.join(
        char for char in normalized
        if not unicodedata.combining(char)
    )
    return plain_str
 
def clean_name(name):
    # Use the remove_accents function to clean the name
    return remove_accents(name)
 
def deduplicate_names(df, fname_col='Fname', lname_col='Lname'):
    # Create new columns for cleaned names
    df['Cleaned_Fname'] = ''
    df['Cleaned_Lname'] = ''
    
    # Identify records with special characters and clean names
    for index, row in df.iterrows():
        first_name = row[fname_col]
        last_name = row[lname_col]
 
        # Clean names using the remove_accents function
        cleaned_first = clean_name(first_name)
        cleaned_last = clean_name(last_name)
 
        # Assign cleaned names back to the DataFrame
        df.at[index, 'Cleaned_Fname'] = cleaned_first
        df.at[index, 'Cleaned_Lname'] = cleaned_last
 
    return df
 
# Example usage:
data = {
    'Fname': ['John', 'Jöhn', 'Anna-Marie', 'José', 'Mårk', 'Elżbieta', 'André', 'Sümmer', 'TóTPφψλη∞', 'Lúcia', 'AαβΓΔΕζ', 'Róbert', 'C@rol', 'J@ne$', '𝒜𝓇𝒾𝒶'],
    'Lname': ['Doe', 'Dœ', 'O\'Connor', 'García', 'Smith', 'Kowalski', 'Tóth', 'Müller', 'Johnson', 'Fernández', 'Brown', 'Leclerc', 'd\'Angelo', 'Smith123', 'Kim']
}
 
df = pd.DataFrame(data)
 
# Clean the DataFrame
cleaned_df = deduplicate_names(df)
 
# Display the cleaned DataFrame
print(cleaned_df[['Fname', 'Lname', 'Cleaned_Fname', 'Cleaned_Lname']])


         Fname      Lname Cleaned_Fname Cleaned_Lname
0         John        Doe          John           Doe
1         Jöhn         Dœ          John            Dœ
2   Anna-Marie   O'Connor    Anna-Marie      O'Connor
3         José     García          Jose        Garcia
4         Mårk      Smith          Mark         Smith
5     Elżbieta   Kowalski      Elzbieta      Kowalski
6        André       Tóth         Andre          Toth
7       Sümmer     Müller        Summer        Muller
8    TóTPφψλη∞    Johnson     ToTPφψλη∞       Johnson
9        Lúcia  Fernández         Lucia     Fernandez
10     AαβΓΔΕζ      Brown       AαβΓΔΕζ         Brown
11      Róbert    Leclerc        Robert       Leclerc
12       C@rol   d'Angelo         C@rol      d'Angelo
13       J@ne$   Smith123         J@ne$      Smith123
14        𝒜𝓇𝒾𝒶        Kim          Aria           Kim


In [32]:
import pandas as pd
import unicodedata
import re

def remove_accents(input_str):
    # Normalize the string to its decomposed form (NFKD)
    normalized = unicodedata.normalize('NFKD', input_str)
    # Build a new string without the accent characters (combining marks)
    plain_str = ''.join(
        char for char in normalized
        if not unicodedata.combining(char)
    )
    return plain_str

def clean_name(name):
    # Use the remove_accents function to clean the name
    cleaned_name = remove_accents(name)
    # Remove non-alphanumeric characters
    cleaned_name = re.sub(r'[^a-zA-Z0-9\s]', '', cleaned_name)
    return cleaned_name

def deduplicate_names(df, fname_col='Fname', lname_col='Lname'):
    # Create new columns for cleaned names
    df['Cleaned_Fname'] = ''
    df['Cleaned_Lname'] = ''
    
    # Identify records with special characters and clean names
    for index, row in df.iterrows():
        first_name = row[fname_col]
        last_name = row[lname_col]

        # Clean names using the clean_name function
        cleaned_first = clean_name(first_name)
        cleaned_last = clean_name(last_name)

        # Assign cleaned names back to the DataFrame
        df.at[index, 'Cleaned_Fname'] = cleaned_first
        df.at[index, 'Cleaned_Lname'] = cleaned_last

    return df

# Example usage:
data = {
    'Fname': ['John', 'Jöhn', 'Anna-Marie', 'José', 'Mårk', 'Elżbieta', 'André', 'Sümmer', 'TóTPφψλη∞', 'Lúcia', 'AαβΓΔΕζ', 'Róbert', 'C@rol', 'J@ne$', '𝒜𝓇𝒾𝒶'],
    'Lname': ['Doe', 'Dœ', "O'Connor", 'García', 'Smith', 'Kowalski', 'Tóth', 'Müller', 'Johnson', 'Fernández', 'Brown', 'Leclerc', "d'Angelo", 'Smith123', 'Kim']
}

df = pd.DataFrame(data)

# Clean the DataFrame
cleaned_df = deduplicate_names(df)

# Display the cleaned DataFrame
print(cleaned_df[['Fname', 'Lname', 'Cleaned_Fname', 'Cleaned_Lname']])

         Fname      Lname Cleaned_Fname Cleaned_Lname
0         John        Doe          John           Doe
1         Jöhn         Dœ          John             D
2   Anna-Marie   O'Connor     AnnaMarie       OConnor
3         José     García          Jose        Garcia
4         Mårk      Smith          Mark         Smith
5     Elżbieta   Kowalski      Elzbieta      Kowalski
6        André       Tóth         Andre          Toth
7       Sümmer     Müller        Summer        Muller
8    TóTPφψλη∞    Johnson          ToTP       Johnson
9        Lúcia  Fernández         Lucia     Fernandez
10     AαβΓΔΕζ      Brown             A         Brown
11      Róbert    Leclerc        Robert       Leclerc
12       C@rol   d'Angelo          Crol       dAngelo
13       J@ne$   Smith123           Jne      Smith123
14        𝒜𝓇𝒾𝒶        Kim          Aria           Kim


In [12]:
import pandas as pd
import re

# Sample dataset
data = {
    'Firstname': ['A', '', 'A', 'Adam', 'A', 'A', 'A', 'A', 'Adam', 'Adam', 'A', 'A', 'A'],
    'Last name': ['Smith', 'Smith', 'S', 'S', 'Smith', 'Smith', 'Smith', 'S', 'S', 'Smith', 'S', 'S', 'Smith'],
    'Email': [
        'a.smith@domain.com',
        'adam.smith@domain.com',
        'adam_smith@domain.com',
        'adam-smith@domain.com',
        'asmith@domain.com',
        'adamsmith@domain.com',
        'asmith@domain.com',
        'adam_smith@domain.com',
        'adam-smith@domain.com',
        'adamsmith@domain.com',
        'adamsmith@domain.com',
        'a.smith@domain.com',
        'adamsmith@domain.com'
    ]
}

# Create DataFrame
df = pd.read_excel("./data/Fname-Lname Extraction data.xlsx")

# Function to extract Fname and Lname from email
def extract_names(row):
    email = row['Email']
    first_name = row['First Name']
    last_name = row['Last Name']

    # Check if there's a separator in the email
    if re.search(r'[._-]', email):
        # Case 1: Extract based on separators
        patterns = [
            r'(?P<Fname>[A-Za-z])\.(?P<Lname>[A-Za-z]+)',  # Pattern 1: F.Lname
            r'(?P<Fname>[A-Za-z]+)\.(?P<Lname>[A-Za-z]+)',  # Pattern 2: Fname.Lname
            r'(?P<Fname>[A-Za-z]+)_(?P<Lname>[A-Za-z]+)',  # Pattern 3: Fname_Lname
            r'(?P<Fname>[A-Za-z]+)-(?P<Lname>[A-Za-z]+)',  # Pattern 4: Fname-Lname
        ]

        for pattern in patterns:
            match = re.match(pattern, email)
            if match:
                extracted_fname = match.group('Fname')
                extracted_lname = match.group('Lname')
                # Fill only if original names are empty or single character
                if (len(str(first_name)) < 2 and extracted_fname) or first_name == '':
                    first_name = extracted_fname
                if (len(str(last_name)) < 2 and extracted_lname) or last_name == '':
                    last_name = extracted_lname
                return first_name, last_name

    else:
        # Case 2: No separator in the email
        email_name = email.split('@')[0]  # Get the part before '@'

        # Case 2A
        if len(first_name) > 2 and len(last_name) > 2:
            if first_name in email_name:
                first_name = first_name
            if last_name in email_name:
                last_name = last_name

        # Case 2B
        if len(first_name) <= 1 or len(last_name) <= 1:
            if len(first_name) == 1 and email_name.startswith(first_name.lower()):
                if len(last_name) > 1 and last_name in email_name:
                    last_name = last_name
            # If both names are single characters or empty, leave as is
            if len(first_name) <= 1 and len(last_name) <= 1:
                return first_name, last_name

        # Extract names from email if they are not already complete
        if len(first_name) < 2 or len(last_name) < 2:
            combined_name = re.sub(r'[^a-zA-Z]', '', email_name)  # Remove any non-alphabetic characters
            if len(first_name) < 2 and combined_name:
                first_name = combined_name[0]
            if len(last_name) < 2 and len(combined_name) > 1:
                last_name = combined_name[1:]

    return first_name, last_name

# Apply the extraction function
df[['Extracted Fname', 'Extracted Lname']] = df.apply(extract_names, axis=1, result_type='expand')

# Display the DataFrame with extracted names
print(df[['First Name', 'Last Name', 'Email', 'Extracted Fname', 'Extracted Lname']])
df.to_csv("Fname-Lname_output_new_logic.csv", index=False)

   First Name Last Name                   Email Extracted Fname  \
0           A     Smith     a.smith@domain.com                a   
1         NaN     Smith  adam.smith@domain.com              NaN   
2           A         S  adam_smith@domain.com             adam   
3        Adam         S  adam-smith@domain.com             Adam   
4           A         S  adam_smith@domain.com             adam   
5        Adam         S  adam-smith@domain.com             Adam   
6        Adam       NaN   smith-adam@domain.com            Adam   
7        Adam         S      asmith@domain.com             Adam   
8           A     Smith   adamsmith@domain.com                A   
9           A     Smith      asmith@domain.com                A   
10       Adam     Smith   adamsmith@domain.com             Adam   
11       Adam         S   adamsmith@domain.com             Adam   
12          A         S   adamsmith@domain.com                A   

   Extracted Lname  
0            Smith  
1            Smith 

In [ ]:
import pandas as pd
import re

def extract_name_from_email(email):
    patterns = [
        r"^([a-zA-Z])\.([a-zA-Z]+)@",  # First letter of first name . last name
        r"^([a-zA-Z]+)\.([a-zA-Z]+)@",  # First name . last name
        r"^([a-zA-Z]+)_([a-zA-Z]+)@",  # First name _ last name
        r"^([a-zA-Z]+)-([a-zA-Z]+)@",  # First name - last name
    ]
    
    for pattern in patterns:
        match = re.match(pattern, email)
        if match:
            return match.groups() if len(match.groups()) == 2 else (match.group(1), "")
    
    return None, None

def process_file(file_path):
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith('.xlsx'):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please use CSV or Excel.")
    
    if not {'First Name', 'Last Name', 'Email'}.issubset(df.columns):
        raise ValueError("Missing required columns: 'FirstName', 'LastName', 'Email'")
    
    for index, row in df.iterrows():
        first_name, last_name, email = row['First Name'], row['Last Name'], row['Email']
        
        if pd.isna(first_name) or pd.isna(last_name) or len(str(first_name)) <= 2 or len(str(last_name)) <= 2:
            extracted_first, extracted_last = extract_name_from_email(email)
            
            if (pd.isna(first_name) or len(str(first_name)) <= 2) and extracted_first:
                df.at[index, 'First Name'] = extracted_first
            if (pd.isna(last_name) or len(str(last_name)) <= 2) and extracted_last:
                df.at[index, 'Last Name'] = extracted_last
    
    output_path = "cleaned_" + file_path.split('/')[-1]
    df.to_csv("Fname-Lname Extraction data 1.csv", index=False)
    print(f"Processed file saved as: {output_path}")

# Example usage
file_path = "./data/Fname-Lname Extraction data 1.xlsx"  # Replace with actual file path
process_file(file_path)


Processed file saved as: cleaned_Fname-Lname Extraction data 1.xlsx


In [63]:
# Final Code for Email Extraction for sepratation of First Name and Last Name
import pandas as pd
import re

def extract_name_from_email(email):
    patterns = [
        r"^([a-zA-Z])\.([a-zA-Z]+)@",  # First letter of first name . last name
        r"^([a-zA-Z]+)\.([a-zA-Z]+)@",  # First name . last name
        r"^([a-zA-Z]+)_([a-zA-Z]+)@",  # First name _ last name
        r"^([a-zA-Z]+)-([a-zA-Z]+)@",  # First name - last name
        r"^([a-zA-Z]+)-([a-zA-Z]+)@",  # Last name - first name
        r"^([a-zA-Z])([a-zA-Z]+)@"
    ]
    
    for pattern in patterns:
        match = re.match(pattern, email)
        if match:
            return match.groups() if len(match.groups()) == 2 else (match.group(1), "")
    
    return None, None

def process_file(file_path):
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith('.xlsx'):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please use CSV or Excel.")
    
    if not {'First Name', 'Last Name', 'Email'}.issubset(df.columns):
        raise ValueError("Missing required columns: 'FirstName', 'LastName', 'Email'")
    
    for index, row in df.iterrows():
        first_name, last_name, email = row['First Name'], row['Last Name'], row['Email']
        
        if pd.isna(first_name) or pd.isna(last_name) or len(str(first_name)) < 2 or len(str(last_name)) < 2:
            extracted_first, extracted_last = extract_name_from_email(email)
            
            if extracted_first and extracted_last:
                if isinstance(first_name, str) and first_name and first_name[0].lower() == extracted_last[0].lower():
                    if len(str(df.at[index, 'First Name']))<= len(extracted_last):
                        df.at[index, 'First Name'] = extracted_last
                    if len(str(df.at[index, 'Last Name']))<= len(extracted_last):
                        df.at[index, 'Last Name'] = extracted_first
                elif isinstance(last_name, str) and last_name and last_name[0].lower() == extracted_first[0].lower():
                    if len(str(df.at[index, 'First Name']))<= len(extracted_first):
                        df.at[index, 'First Name'] = extracted_first
                    if len(str(df.at[index, 'Last Name']))< len(extracted_first):
                        df.at[index, 'Last Name'] = extracted_last
                else:
                    if (pd.isna(first_name) or len(str(first_name)) <= 2):
                        df.at[index, 'First Name'] = extracted_first
                    if (pd.isna(last_name) or len(str(last_name)) <= 2):
                        df.at[index, 'Last Name'] = extracted_last
    
    output_path = "cleaned_" + file_path.split('/')[-1]
    df.to_csv("Fname-Lname Extraction data 1.csv", index=False)
    print(f"Processed file saved as: {output_path}")

# Example usage
file_path = "./data/Fname-Lname Extraction data 1.xlsx"  # Replace with actual file path
process_file(file_path)

Processed file saved as: cleaned_Fname-Lname Extraction data 1.xlsx


In [ ]:
import pandas as pd
import re

def extract_name_from_email(email):
    patterns = [
        r"^([a-zA-Z])\.([a-zA-Z]+)@",  # First letter of first name . last name
        r"^([a-zA-Z]+)\.([a-zA-Z]+)@",  # First name . last name
        r"^([a-zA-Z]+)_([a-zA-Z]+)@",  # First name _ last name
        r"^([a-zA-Z]+)-([a-zA-Z]+)@",  # First name - last name
        r"^([a-zA-Z]+)-([a-zA-Z]+)@",  # Last name - first name
    ]
    
    for pattern in patterns:
        match = re.match(pattern, email)
        if match:
            return match.groups() if len(match.groups()) == 2 else (match.group(1), "")
    
    return None, None

def process_file(file_path):
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith('.xlsx'):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please use CSV or Excel.")
    
    if not {'First Name', 'Last Name', 'Email'}.issubset(df.columns):
        raise ValueError("Missing required columns: 'First Name', 'Last Name', 'Email'")
    
    for index, row in df.iterrows():
        first_name, last_name, email = row['First Name'], row['Last Name'], row['Email']
        
        if pd.isna(first_name) or pd.isna(last_name) or len(str(first_name)) < 2 or len(str(last_name)) < 2:
            extracted_first, extracted_last = extract_name_from_email(email)
            
            if extracted_first and extracted_last:
                if isinstance(first_name, str) and first_name and first_name[0].lower() == extracted_last[0].lower():
                    df.at[index, 'First Name'] = extracted_last
                    df.at[index, 'Last Name'] = extracted_first
                elif isinstance(last_name, str) and last_name and last_name[0].lower() == extracted_first[0].lower():
                    df.at[index, 'First Name'] = extracted_first
                    df.at[index, 'Last Name'] = extracted_last
                else:
                    if (pd.isna(first_name) or len(str(first_name)) < 2) or (len(extracted_first) > len(first_name)):
                        df.at[index, 'First Name'] = extracted_first
                    if (pd.isna(last_name) or len(str(last_name)) < 2) or (len(extracted_last) > len(last_name)):
                        df.at[index, 'Last Name'] = extracted_last
    
    output_path = "cleaned_" + file_path.split('/')[-1]
    df.to_csv("Fname-Lname Extraction data 1.csv", index=False)
    print(f"Processed file saved as: {output_path}")

# Example usage
file_path = "./data/Fname-Lname Extraction data 1.xlsx"  # Replace with actual file path
process_file(file_path)

Processed file saved as: cleaned_Fname-Lname Extraction data 1.xlsx


In [41]:
import pandas as pd
import re
 
# Sample dataset
data = {
   'Firstname': ['A', '', 'A', 'Adam', 'A', 'Adam', 'Adam', 'A', 'Adam', 'Adam', 'A', 'A', 'A'],
   'Last name': ['Smith', 'Smith', 'S', 'S', 'Smith', 'Smith', 'Smith', 'S', 'S', 'Smith', 'S', 'S', 'Smith'],
   'Email': [
       'a.smith@domain.com',
       'adam.smith@domain.com',
       'adam_smith@domain.com',
       'adam-smith@domain.com',
       'asmith@domain.com',
       'adamsmith@domain.com',
       'asmith@domain.com',
       'adam_smith@domain.com',
       'adam-smith@domain.com',
       'adamsmith@domain.com',
       'adamsmith@domain.com',
       'a.smith@domain.com',
       'adamsmith@domain.com'
   ]
}
 
# Create DataFrame
df = pd.read_excel("./data/Fname-Lname Extraction data 1.xlsx")
 
# Function to extract Fname and Lname from email
def extract_names(row):
   email = row['Email']
   first_name = row['First Name']
   last_name = row['Last Name']
 
   # Check if there's a separator in the email
   if re.search(r'[._-]', email):
       # Case 1: Extract based on separators
       patterns = [
          r'(?P<Fname>[A-Za-z])\.(?P<Lname>[A-Za-z]+)',  # Pattern 1: F.Lname
          r'(?P<Fname>[A-Za-z]+)\.(?P<Lname>[A-Za-z]+)',  # Pattern 2: Fname.Lname
          r'(?P<Fname>[A-Za-z]+)_(?P<Lname>[A-Za-z]+)',  # Pattern 3: Fname_Lname
          r'(?P<Fname>[A-Za-z]+)-(?P<Lname>[A-Za-z]+)',  # Pattern 4: Fname-Lname
       ]
 
       for pattern in patterns:
           match = re.match(pattern, email)
           if match:
               extracted_fname = match.group('Fname')
               extracted_lname = match.group('Lname')
               # Fill only if original names are empty or single character
               if (len(str(first_name)) < 2 and extracted_fname) or first_name == '':
                   first_name = extracted_fname
               if (len(str(last_name)) < 2 and extracted_lname) or last_name == '':
                   last_name = extracted_lname
               return first_name, last_name
 
   else:
       # Case 2: No separator in the email
       email_name = email.split('@')[0]  # Get the part before '@'
 
       # Case 2A
       if len(str(first_name)) > 2 and len(str(last_name)) > 2:
           if first_name in email_name:
               first_name = first_name
           if last_name in email_name:
               last_name = last_name
 
       # Case 2B
       if len(str(first_name)) <= 1 or len(str(last_name)) <= 1:
           if len(str(first_name)) == 1 and email_name.startswith(first_name.lower()):
               if len(str(last_name)) > 1 and last_name in email_name:
                   last_name = last_name
           # If both names are single characters or empty, leave as is
           if len(str(first_name)) <= 1 and len(str(last_name)) <= 1:
               return first_name, last_name
 
       # Extract names from email if they are not already complete
       if len(str(first_name)) < 2 or len(str(last_name)) < 2:
           combined_name = re.sub(r'[^a-zA-Z]', '', email_name)  # Remove any non-alphabetic characters
           if len(str(first_name)) < 2 and combined_name:
               first_name = combined_name[0]
           if len(str(last_name)) < 2 and len(combined_name) > 1:
               last_name = combined_name[1:]
 
   return first_name, last_name
 
# Apply the extraction function
df[['Extracted Fname', 'Extracted Lname']] = df.apply(extract_names, axis=1, result_type='expand')
 
# Display the DataFrame with extracted names
print(df[['First Name', 'Last Name', 'Email', 'Extracted Fname', 'Extracted Lname']])

   First Name Last Name                   Email Extracted Fname  \
0           A     Smith     a.smith@domain.com                a   
1         NaN       NaN   adam.smith@domain.com             NaN   
2         NaN     Smith  adam.smith@domain.com              NaN   
3           A         S  adam_smith@domain.com             adam   
4        Adam         S  adam-smith@domain.com             Adam   
5           A         S  adam_smith@domain.com             adam   
6           A         S  smith.adam@domain.com            smith   
7        Adam         S  adam-smith@domain.com             Adam   
8        Adam       NaN   smith-adam@domain.com            Adam   
9        Adam         S      smith.a@domain.com            Adam   
10       Adam         S      asmith@domain.com             Adam   
11       Adam         S       smith@domain.com             Adam   
12          A     Smith        adam@domain.com                A   
13          A     Smith   adamsmith@domain.com                

In [45]:
import pandas as pd
import re

def extract_name_from_email(email):
    patterns = [
        r"^([a-zA-Z])\.([a-zA-Z]+)@",  # First letter of first name . last name
        r"^([a-zA-Z]+)\.([a-zA-Z]+)@",  # First name . last name
        r"^([a-zA-Z]+)_([a-zA-Z]+)@",  # First name _ last name
        r"^([a-zA-Z]+)-([a-zA-Z]+)@",  # First name - last name
        r"^([a-zA-Z]+)-([a-zA-Z]+)@",  # Last name - first name
        r"^([a-zA-Z])([a-zA-Z]+)@",    # First letter of first name + last name
    ]
    
    for pattern in patterns:
        match = re.match(pattern, email)
        if match:
            if len(match.groups()) == 2:
                first, last = match.groups()
                if len(first) == 1 and len(last) > 1:
                    return first + last[0], last[1:]
                return first, last
            else:
                return match.group(1), ""
    
    return None, None

def process_file(file_path):
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith('.xlsx'):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please use CSV or Excel.")
    
    if not {'First Name', 'Last Name', 'Email'}.issubset(df.columns):
        raise ValueError("Missing required columns: 'First Name', 'Last Name', 'Email'")
    
    for index, row in df.iterrows():
        first_name, last_name, email = row['First Name'], row['Last Name'], row['Email']
        
        if pd.isna(first_name) or pd.isna(last_name) or len(str(first_name)) < 2 or len(str(last_name)) < 2:
            extracted_first, extracted_last = extract_name_from_email(email)
            
            if extracted_first and extracted_last:
                if isinstance(first_name, str) and first_name and first_name[0].lower() == extracted_last[0].lower():
                    if len(str(df.at[index, 'First Name'])) <= len(extracted_last):
                        df.at[index, 'First Name'] = extracted_last
                    if len(str(df.at[index, 'Last Name'])) <= len(extracted_last):
                        df.at[index, 'Last Name'] = extracted_first
                elif isinstance(last_name, str) and last_name and last_name[0].lower() == extracted_first[0].lower():
                    if len(str(df.at[index, 'First Name'])) <= len(extracted_first):
                        df.at[index, 'First Name'] = extracted_first
                    if len(str(df.at[index, 'Last Name'])) < len(extracted_first):
                        df.at[index, 'Last Name'] = extracted_last
                else:
                    if pd.isna(first_name) or len(str(first_name)) <= 2:
                        df.at[index, 'First Name'] = extracted_first
                    if pd.isna(last_name) or len(str(last_name)) <= 2:
                        df.at[index, 'Last Name'] = extracted_last
    
    output_path = "cleaned_" + file_path.split('/')[-1]
    df.to_csv("Fname-Lname Extraction data 1.csv", index=False)
    print(f"Processed file saved as: {output_path}")

# Example usage
file_path = "./data/Fname-Lname Extraction data 1.xlsx"  # Replace with actual file path
process_file(file_path)

Processed file saved as: cleaned_Fname-Lname Extraction data 1.xlsx


In [ ]:
import pandas as pd

def process_data(df):
    def extract_names(row):
        # Extract components from the row
        fname = row['Firstname'].lower()  # Convert to lowercase
        lname = row['Last name'].lower()   # Convert to lowercase
        email = row['Email'].lower()        # Convert to lowercase
        
        # Check if Lname is a single letter and Email is valid
        if len(lname) == 1 and '@' in email:
            # Extract the part before the '@' from the email
            extracted_name = email.split('@')[0]  # e.g., "wharris"
            
            # Check if extracted_name starts with the first letter of Fname followed by Lname
            if extracted_name[0] == fname[0] and extracted_name[1:].startswith(lname): 
                # Update Last name with the full last name from email
                row['Last name'] = extracted_name[1:]# Capitalize the last name
                return row

        return row  # Return original row if no conditions are met

    # Apply the processing function to each row in the DataFrame
    df = df.apply(extract_names, axis=1)
    return df

# Create DataFrame
df = pd.read_excel("./data/Data.xlsx")

# Process the DataFrame
result_df = process_data(df)

# Output the processed DataFrame
print(result_df)


  Firstname Last name                 Email
0      Adam     smith     asmith@domain.com
1    Andrew   johnson   ajohnson@domain.com
2   Anthony  williams  awilliams@domain.com
3      Avis     brown     abrown@domain.com
4  Williams    harris     wharris@gmail.com


In [ ]:
import pandas as pd
import re

def extract_name_from_email(email):
    patterns = [
        r"^([a-zA-Z])\.([a-zA-Z]+)@",  # First letter of first name . last name
        r"^([a-zA-Z]+)\.([a-zA-Z]+)@",  # First name . last name
        r"^([a-zA-Z]+)_([a-zA-Z]+)@",  # First name _ last name
        r"^([a-zA-Z]+)-([a-zA-Z]+)@",  # First name - last name
        r"^([a-zA-Z]+)-([a-zA-Z]+)@",  # Last name - first name
        r"^([a-zA-Z])([a-zA-Z]+)@"
    ]
    
    for pattern in patterns:
        match = re.match(pattern, email)
        if match:
            return match.groups() if len(match.groups()) == 2 else (match.group(1), "")
    
    return None, None

def extract_names(row):
    fname = row['First Name'].lower() if pd.notna(row['First Name']) else ""
    lname = row['Last Name'].lower() if pd.notna(row['Last Name']) else ""
    email = row['Email'].lower()
    
    extracted_first, extracted_last = extract_name_from_email(email)
    
    if extracted_first and extracted_last:
        if pd.isna(row['First Name']) or len(row['First Name']) < 2:
            row['First Name'] = extracted_first.capitalize()
        if pd.isna(row['Last Name']) or len(row['Last Name']) < 2:
            row['Last Name'] = extracted_last.capitalize()
    else:
        if len(lname) == 1 and '@' in email:
            extracted_name = email.split('@')[0]
            if len(extracted_name) > 1 and extracted_name[0] == fname[0] and extracted_name[1:].startswith(lname):
                row['Last Name'] = extracted_name[1:].capitalize()
    
    return row

def process_file(file_path):
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith('.xlsx'):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please use CSV or Excel.")
    
    if not {'First Name', 'Last Name', 'Email'}.issubset(df.columns):
        raise ValueError("Missing required columns: 'First Name', 'Last Name', 'Email'")
    
    df = df.apply(extract_names, axis=1)
    
    output_path = "cleaned_" + file_path.split('/')[-1]
    df.to_csv("Fname-Lname Extraction data 1.csv", index=False)
    print(f"Processed file saved as: {output_path}")

# Example usage
file_path = "./data/Fname-Lname Extraction data 1.xlsx"  # Replace with actual file path
process_file(file_path)

Processed file saved as: cleaned_Fname-Lname Extraction data 1.xlsx


In [ ]:
import pandas as pd
import re

def extract_name_from_email(email):
    patterns = [
        r"^([a-zA-Z])\.([a-zA-Z]+)@",  # First letter of first name . last name
        r"^([a-zA-Z]+)\.([a-zA-Z]+)@",  # First name . last name
        r"^([a-zA-Z]+)_([a-zA-Z]+)@",  # First name _ last name
        r"^([a-zA-Z]+)-([a-zA-Z]+)@",  # First name - last name
        r"^([a-zA-Z]+)-([a-zA-Z]+)@",  # Last name - first name
    ]
    
    for pattern in patterns:
        match = re.match(pattern, email)
        if match:
            return match.groups() if len(match.groups()) == 2 else (match.group(1), "")
    
    return None, None

def extract_names(row):
    fname = row['First Name'].lower() if pd.notna(row['First Name']) else ""
    lname = row['Last Name'].lower() if pd.notna(row['Last Name']) else ""
    email = row['Email'].lower()
    
    extracted_first, extracted_last = extract_name_from_email(email)
    
    if extracted_first and extracted_last:
        if pd.isna(row['First Name']) or len(row['First Name']) < 2:
            row['First Name'] = extracted_first.capitalize()
        if pd.isna(row['Last Name']) or len(row['Last Name']) < 2:
            row['Last Name'] = extracted_last.capitalize()
    else:
        if len(lname) == 1 and '@' in email:
            extracted_name = email.split('@')[0]
            if len(extracted_name) > 1 and extracted_name[0] == fname[0] and extracted_name[1:].startswith(lname):
                row['Last Name'] = extracted_name[1:].capitalize()
    
    return row

def process_file(file_path):
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith('.xlsx'):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please use CSV or Excel.")
    
    if not {'First Name', 'Last Name', 'Email'}.issubset(df.columns):
        raise ValueError("Missing required columns: 'First Name', 'Last Name', 'Email'")
    
    for index, row in df.iterrows():
        first_name, last_name, email = row['First Name'], row['Last Name'], row['Email']
        
        if pd.isna(first_name) or pd.isna(last_name) or len(str(first_name)) < 2 or len(str(last_name)) < 2:
            extracted_first, extracted_last = extract_name_from_email(email)
            
            if extracted_first and extracted_last:
                if isinstance(first_name, str) and first_name and first_name[0].lower() == extracted_last[0].lower():
                    if len(str(df.at[index, 'First Name'])) <= len(extracted_last):
                        df.at[index, 'First Name'] = extracted_last.capitalize()
                    if len(str(df.at[index, 'Last Name'])) <= len(extracted_last):
                        df.at[index, 'Last Name'] = extracted_first.capitalize()
                elif isinstance(last_name, str) and last_name and last_name[0].lower() == extracted_first[0].lower():
                    if len(str(df.at[index, 'First Name'])) <= len(extracted_first):
                        df.at[index, 'First Name'] = extracted_first.capitalize()
                    if len(str(df.at[index, 'Last Name'])) < len(extracted_first):
                        df.at[index, 'Last Name'] = extracted_last.capitalize()
                else:
                    if pd.isna(first_name) or len(str(first_name)) <= 2:
                        df.at[index, 'First Name'] = extracted_first.capitalize()
                    if pd.isna(last_name) or len(str(last_name)) <= 2:
                        df.at[index, 'Last Name'] = extracted_last.capitalize()
            else:
                df.loc[index] = extract_names(row)
    
    output_path = "cleaned_" + file_path.split('/')[-1]
    df.to_csv("Fname-Lname Extraction data 1.csv", index=False)
    print(f"Processed file saved as: {output_path}")

# Example usage
file_path = "./data/Fname-Lname Extraction data 1.xlsx"  # Replace with actual file path
process_file(file_path)

Processed file saved as: cleaned_Fname-Lname Extraction data 1.xlsx


## add email part as well

In [17]:
import src.util.similarity_helper as sim_helper

def group_by_last_name(df, threshold=0.85):
    groups = []
    for index, row in df.iterrows():
        last_name = str(row['LastName'])
        email = str(row.get('Email', ''))  # Ensure email is a string
        if not last_name:  # Check if last_name is empty
            continue  # Ensure last_name is a string
        found_group = False
        for group in groups:
            # Check if the first character of the last name is the same
            if last_name[0] == str(group[0]['LastName'])[0]:
                if sim_helper.is_similar(last_name, str(group[0]['LastName']), threshold) or email == str(group[0].get('Email', '')):
                    group.append(row)
                    found_group = True
                    break
        if not found_group:
            groups.append([row])
    return groups

In [18]:
def refine_by_first_name(groups, threshold=0.65):
    refined_groups = []
    for group in groups:
        refined_group = []
        single_letter_names = []
        empty_first_names = []  # To store contacts with empty first names
        
        for contact in group:
            first_name = str(contact.get('FirstName', ''))  # Ensure first_name is a string
            email = str(contact.get('Email', ''))  # Ensure email is a string
            if not first_name:
                empty_first_names.append(contact)
                continue
            if len(first_name) == 1:
                single_letter_names.append(contact)
                continue
            
            found_subgroup = False
            for subgroup in refined_group:
                if first_name[0] == str(subgroup[0]['FirstName'])[0]:
                    if sim_helper.is_similar(first_name, str(subgroup[0]['FirstName']), threshold) or email == str(subgroup[0].get('Email', '')):
                        subgroup.append(contact)
                        found_subgroup = True
                        break
            if not found_subgroup:
                refined_group.append([contact])
        
        # Integrate single-letter names into the appropriate subgroups
        for contact in single_letter_names:
            first_name = str(contact.get('FirstName', ''))
            email = str(contact.get('Email', ''))
            found_subgroup = False
            for subgroup in refined_group:
                if first_name[0] == str(subgroup[0]['FirstName'])[0] or email == str(subgroup[0].get('Email', '')):
                    subgroup.append(contact)
                    found_subgroup = True
                    break
            if not found_subgroup:
                refined_group.append([contact])
        
        # Add the empty first name contacts as a separate subgroup
        if empty_first_names:
            refined_group.append(empty_first_names)
        
        refined_groups.append(refined_group)
    
    return refined_groups

In [19]:
def assign_ids(refined_groups):
    group_id = 1
    subgroup_id = 1  # Initialize a global counter for SubgroupID
    result = []
    for group in refined_groups:
        for subgroup in group:
            is_duplicate = len(subgroup) > 1
            for contact in subgroup:
                if is_duplicate:
                    contact['GroupID'] = group_id
                    contact['dup_group_id'] = subgroup_id
                else:
                    contact['GroupID'] = group_id
                    contact['dup_group_id'] = 999999  # No subgroup ID for unique records
                contact['IsDuplicate'] = is_duplicate
                result.append(contact)
            if is_duplicate:
                subgroup_id += 1  # Increment the global SubgroupID counter only for duplicates
        group_id += 1
    return result

In [20]:
import pandas as pd
df = pd.read_excel("./data/SouthFiled_Contact_Input-300 records.xlsx")

broad_groups = group_by_last_name(df)

# Refine by first name
refined_groups = refine_by_first_name(broad_groups)

# Assign unique IDs and flag duplicates
processed_contacts = assign_ids(refined_groups)

# Convert processed contacts back to DataFrame
processed_df = pd.DataFrame(processed_contacts)
print(processed_df.head())

           Full Name FirstName  LastName                      Job Title  \
0      Andrew Minkow    Andrew    Minkow                            CFO   
1      Andrew Minkow         A    Minkow                            CFO   
2   Michael Morrison   Michael  Morrison  Managing Director & Principal   
3   Michael Morrison       NaN       NaN  Managing Director & Principal   
12        Paul Traut       NaN       NaN                      Associate   

                    Company    Office Phone                    Email  \
0   Pioneer Power Solutions  (212) 867-1070        aminkow@gmail.com   
1   Pioneer Power Solutions  (212) 867-1070        aminkow@gmail.com   
2                   4655221  (804) 591-2051  mmorrison@matrixcmg.com   
3                   4655221  (804) 591-2051  mmorrison@matrixcmg.com   
12               DH Capital             NaN     PTraut@dhcapital.com   

     Last Contact Date State Contact Type (Entry Only)  \
0  2015-06-12 00:00:00    NJ                       NaN   


In [ ]:
#email 
def assign_ids(refined_groups):
    group_id = 1
    subgroup_id = 1  # Initialize a global counter for SubgroupID
    result = []
    email_to_group = {}  # Dictionary to track email to group mapping

    for group in refined_groups:
        for subgroup in group:
            is_duplicate = len(subgroup) > 1
            for contact in subgroup:
                email = str(contact.get('Email', '')).strip().lower()  # Convert email to string

                if email in email_to_group:
                    contact['GroupID'] = email_to_group[email]['GroupID']
                    contact['dup_group_id'] = email_to_group[email]['dup_group_id']
                    contact['IsDuplicate'] = True
                else:
                    if is_duplicate:
                        contact['GroupID'] = group_id
                        contact['dup_group_id'] = subgroup_id
                        email_to_group[email] = {'GroupID': group_id, 'dup_group_id': subgroup_id}  # Map email to current group and subgroup
                    else:
                        contact['GroupID'] = group_id
                        contact['dup_group_id'] = 999999  # No subgroup ID for unique records
                    contact['IsDuplicate'] = is_duplicate
                result.append(contact)

            if is_duplicate:
                subgroup_id += 1  # Increment the global SubgroupID counter only for duplicates

        group_id += 1

    return result

In [21]:
processed_df.to_csv('./Subset-input data for COntact - Centerbridge - 3k.csv', index=False)

In [ ]:
import pandas as pd
import src.util.log_helper as log_helper
import src.util.helper as helper
logger = log_helper.set_get_logger("company_deduplication",helper.get_logfile_name())

# Load the Excel file
df = pd.read_excel('./data/SouthFiled_Contact_Input-300 records.xlsx', engine='openpyxl')

def group_by_last_name(df, threshold=0.85):
    groups = []
    for index, row in df.iterrows():
        last_name = str(row['LastName'])
        if not last_name:  # Check if last_name is empty
            continue  # Ensure last_name is a string
        found_group = False
        for group in groups:
            # Check if the first character of the last name is the same
            if last_name[0] == str(group[0]['LastName'])[0]:
                if sim_helper.is_similar(last_name, str(group[0]['LastName']), threshold):
                    group.append(row)
                    found_group = True
                    break
        if not found_group:
            groups.append([row])
    return groups

def refine_by_first_name(groups, threshold=0.65):
    refined_groups = []
    for group in groups:
        refined_group = []
        single_letter_names = []
        empty_first_names = []  # To store contacts with empty first names
        
        for contact in group:
            first_name = str(contact.get('FirstName', ''))  # Ensure first_name is a string
            if not first_name:
                empty_first_names.append(contact)
                continue
            if len(first_name) == 1:
                single_letter_names.append(contact)
                continue
            
            found_subgroup = False
            for subgroup in refined_group:
                if first_name[0] == str(subgroup[0]['FirstName'])[0]:
                    if sim_helper.is_similar(first_name, str(subgroup[0]['FirstName']), threshold):
                        subgroup.append(contact)
                        found_subgroup = True
                        break
            if not found_subgroup:
                refined_group.append([contact])
        
        # Integrate single-letter names into the appropriate subgroups
        for contact in single_letter_names:
            first_name = str(contact.get('FirstName', ''))
            found_subgroup = False
            for subgroup in refined_group:
                if first_name[0] == str(subgroup[0]['FirstName'])[0]:
                    subgroup.append(contact)
                    found_subgroup = True
                    break
            if not found_subgroup:
                refined_group.append([contact])
        
        # Add the empty first name contacts as a separate subgroup
        if empty_first_names:
            refined_group.append(empty_first_names)
        
        refined_groups.append(refined_group)
    
    return refined_groups


#email 
def assign_ids(refined_groups):
    group_id = 1
    subgroup_id = 1  # Initialize a global counter for SubgroupID
    result = []
    email_to_group = {}  # Dictionary to track email to group mapping

    for group in refined_groups:
        for subgroup in group:
            is_duplicate = len(subgroup) > 1
            for contact in subgroup:
                email = str(contact.get('Email', '')).strip().lower()  # Convert email to string

                if email in email_to_group:
                    contact['GroupID'] = email_to_group[email]['GroupID']
                    contact['dup_group_id'] = email_to_group[email]['dup_group_id']
                    contact['IsDuplicate'] = True
                else:
                    if is_duplicate:
                        contact['GroupID'] = group_id
                        contact['dup_group_id'] = subgroup_id
                        email_to_group[email] = {'GroupID': group_id, 'dup_group_id': subgroup_id}  # Map email to current group and subgroup
                    else:
                        contact['GroupID'] = group_id
                        contact['dup_group_id'] = 999999  # No subgroup ID for unique records
                    contact['IsDuplicate'] = is_duplicate
                result.append(contact)

            if is_duplicate:
                subgroup_id += 1  # Increment the global SubgroupID counter only for duplicates

        group_id += 1

    return result

# Email Grouping 
def email_grouping(df):
    counter = 1
    groups = []
    for index, row in df.iterrows():
        email = str(row['Email']).lower()
        
        if not email:  # Check if email is empty
            continue  # Ensure email is a string
        found_group = False
        for group in groups:
            #print(f"counter value{counter}")
            if email == str(group[0]['Email']).lower():
                print(f"Email: {email}")
                print(f"Row: {row['Email']}")
                print(f"Row: {row['FirstName']}")
                print(f"Row: {row['LastName']}")
                print(f"Row: {row['dup_group_id']}")
                group.append(row)
                found_group = True
                break
            counter=counter+1
        if not found_group:
            groups.append([row])
    return groups

# Group contacts by last name and email
groups = group_by_last_name(df)

# Refine groups by first name and email
refined_groups = refine_by_first_name(groups)

# Assign IDs to refined groups
final_result = assign_ids(refined_groups)
# Convert the result to a DataFrame and save to Excel
final_df = pd.DataFrame(final_result)
#email_grouping = email_grouping(final_df)

grouped = final_df.groupby('Email')

print("Grouped Records (Only groups with more than 1 record):")
group_no = 1
for email, group_data in grouped:
    # Check if the group contains more than one record
    if len(group_data) > 1:
        print(f"\nGroup {group_no}: Email: {email}")
        for _, row in group_data.iterrows():
            print(f"  {row['FirstName']} {row['LastName']} - {row['Email']}")
        group_no += 1

# Save the final DataFrame to an Excel file

Grouped Records (Only groups with more than 1 record):

Group 1: Email: PTraut@dhcapital.com
  nan nan - PTraut@dhcapital.com
  Paul Traut - PTraut@dhcapital.com

Group 2: Email: aminkow@gmail.com
  Andrew Minkow - aminkow@gmail.com
  A Minkow - aminkow@gmail.com

Group 3: Email: dramsey@globalmna.com
  nan nan - dramsey@globalmna.com
  Dustin Ramsey - dramsey@globalmna.com

Group 4: Email: gladdish@hydeparkcapital.com
  Matt Gladdish - gladdish@hydeparkcapital.com
  M Gladdish - gladdish@hydeparkcapital.com

Group 5: Email: mmorrison@matrixcmg.com
  Michael Morrison - mmorrison@matrixcmg.com
  nan nan - mmorrison@matrixcmg.com


In [54]:
import pandas as pd

# Sample data with columns FirstName, LastName, and Email
data = [
    {"FirstName": "Alice",   "LastName": "Smith",    "Email": "alice@example.com"},
    {"FirstName": "Bob",     "LastName": "Johnson",  "Email": "bob@example.com"},
    {"FirstName": "Charlie", "LastName": "Williams", "Email": "alice@example.com"},
    {"FirstName": "David",   "LastName": "Brown",    "Email": "david@example.com"},
    {"FirstName": "Eva",     "LastName": "Jones",    "Email": "bob@example.com"}
]

# Create DataFrame
df = pd.DataFrame(data)

# Group records by Email
grouped = df.groupby('Email')

# Iterate over the groups and print the output
print("Grouped Records:")
for group_no, (email, group_data) in enumerate(grouped, start=1):
    print(f"\nGroup {group_no}: Email: {email}")
    for _, row in group_data.iterrows():
        print(f"  {row['FirstName']} {row['LastName']} - {row['Email']}")


Grouped Records:

Group 1: Email: alice@example.com
  Alice Smith - alice@example.com
  Charlie Williams - alice@example.com

Group 2: Email: bob@example.com
  Bob Johnson - bob@example.com
  Eva Jones - bob@example.com

Group 3: Email: david@example.com
  David Brown - david@example.com


In [ ]:
def group_by_last_name(df, threshold=0.85):
    groups = []
    for index, row in df.iterrows():
        last_name = str(row['LastName'])
        if not last_name:  # Check if last_name is empty
            break  # Ensure last_name is a string
        found_group = False
        for group in groups:
            # Check if the first character of the last name is the same
            if last_name[0] == str(group[0]['LastName'])[0]:
                if sim_helper.is_similar(last_name, str(group[0]['LastName']), threshold):
                    group.append(row)
                    found_group = True
                    break
        if not found_group:
            groups.append([row])
    return groups

def refine_by_first_name(groups, threshold=0.65):
    refined_groups = []
    for group in groups:
        refined_group = []
        single_letter_names = []
        empty_first_names = []  # To store contacts with empty first names
        
        for contact in group:
            first_name = str(contact.get('FirstName', ''))  # Ensure first_name is a string
            if not first_name:
                empty_first_names.append(contact)
                continue
            if len(first_name) == 1:
                single_letter_names.append(contact)
                continue
            
            found_subgroup = False
            for subgroup in refined_group:
                if first_name[0] == str(subgroup[0]['FirstName'])[0]:
                    if sim_helper.is_similar(first_name, str(subgroup[0]['FirstName']), threshold):
                        subgroup.append(contact)
                        found_subgroup = True
                        break
            if not found_subgroup:
                refined_group.append([contact])
        
        # Integrate single-letter names into the appropriate subgroups
        for contact in single_letter_names:
            first_name = str(contact.get('FirstName', ''))
            found_subgroup = False
            for subgroup in refined_group:
                if first_name[0] == str(subgroup[0]['FirstName'])[0]:
                    subgroup.append(contact)
                    found_subgroup = True
                    break
            if not found_subgroup:
                refined_group.append([contact])
        
        # Add the empty first name contacts as a separate subgroup
        if empty_first_names:
            refined_group.append(empty_first_names)
        
        refined_groups.append(refined_group)
    
    return refined_groups

def assign_ids(refined_groups):
    group_id = 1
    subgroup_id = 1  # Initialize a global counter for SubgroupID
    result = []
    for group in refined_groups:
        for subgroup in group:
            is_duplicate = len(subgroup) > 1
            for contact in subgroup:
                if is_duplicate:
                    contact['GroupID'] = group_id
                    contact['dup_group_id'] = subgroup_id
                else:
                    contact['GroupID'] = group_id
                    contact['dup_group_id'] = 999999  # No subgroup ID for unique records
                contact['IsDuplicate'] = is_duplicate
                result.append(contact)
            if is_duplicate:
                subgroup_id += 1  # Increment the global SubgroupID counter only for duplicates
        group_id += 1
    return result,subgroup_id


In [2]:
import pandas as pd
import src.util.similarity_helper as sim_helper

df = pd.read_excel('./data/SouthFiled_Contact_Input-300 records.xlsx', engine='openpyxl')
# Group contacts by last name and email
groups = group_by_last_name(df)

# Refine groups by first name and email
refined_groups = refine_by_first_name(groups)

# Assign IDs to refined groups
final_result,subgroup_id = assign_ids(refined_groups)
final_df = pd.DataFrame(final_result)

In [76]:
final_df.columns

Index(['Full Name', 'FirstName', 'LastName', 'Job Title', 'Company',
       'Office Phone', 'Email', 'Last Contact Date', 'State',
       'Contact Type (Entry Only)', 'Last Activity', 'dup_group_id', 'GroupID',
       'IsDuplicate'],
      dtype='object')

In [3]:
final_df.to_csv('./Suebset-input data for COntact - Centerbridge - 3k.csv', index=False)

In [3]:
grouped = final_df.groupby('Email')
new_group_counter = subgroup_id

for email, group_data in grouped:
    if len(group_data) > 1:
        # Check for any non-default (non "99999") group id in this email group
        non_default_ids = group_data[group_data['dup_group_id'] != "999999"]['dup_group_id'].unique()
        if len(non_default_ids) > 0:
            # Use the first non-default group id found
            assigned_id = non_default_ids[0]
        else:
            # Generate a new group id if all are "99999"
            assigned_id = f"new_group_{new_group_counter}"
            new_group_counter += 1
        
        # Update records in the original DataFrame: assign the chosen id to rows with "99999"
        mask = (final_df['Email'] == email) & (final_df['dup_group_id'] == "999999")
        final_df.loc[mask, 'dup_group_id'] = assigned_id

# Print out only groups with more than one record (grouped by email)
print("Updated Groups (only groups with more than 1 record):")
for email, group_data in final_df.groupby('Email'):
    if len(group_data) > 1:
        print(f"\nEmail: {email}")
        for idx, row in group_data.iterrows():
            print(f"  {row['FirstName']} {row['LastName']} - {row['Email']} - Dup_group_id: {row['dup_group_id']}")



Updated Groups (only groups with more than 1 record):

Email: PTraut@dhcapital.com
  nan nan - PTraut@dhcapital.com - Dup_group_id: 2
  Paul Traut - PTraut@dhcapital.com - Dup_group_id: 999999

Email: aminkow@gmail.com
  Andrew Minkow - aminkow@gmail.com - Dup_group_id: 1
  A Minkow - aminkow@gmail.com - Dup_group_id: 1

Email: dramsey@globalmna.com
  nan nan - dramsey@globalmna.com - Dup_group_id: 2
  Dustin Ramsey - dramsey@globalmna.com - Dup_group_id: 999999

Email: gladdish@hydeparkcapital.com
  Matt Gladdish - gladdish@hydeparkcapital.com - Dup_group_id: 3
  M Gladdish - gladdish@hydeparkcapital.com - Dup_group_id: 3

Email: mmorrison@matrixcmg.com
  Michael Morrison - mmorrison@matrixcmg.com - Dup_group_id: 999999
  nan nan - mmorrison@matrixcmg.com - Dup_group_id: 2
